# NV-Retriever 방법론 실험

1. Teacher - Base Model 선정
    - Teacher 후보 모델 (한국어 / 영어)
        - https://huggingface.co/DeepMount00/Llama-3.1-8b-ITA
        - https://huggingface.co/AIDX-ktds/ktdsbaseLM-v0.12-based-on-openchat3.5
    - Base 후보 모델 (한국어 / 영어)
        - https://huggingface.co/intfloat/multilingual-e5-large

2. Dataset 구성
    - 사용할 데이터셋 리스트업 (한국어 / 영어)
        - 한국어
            - (QA) nayohan/Sujet-Finance-Instruct-177k-ko
            - (QA) BCCard/BCCard-Finance-Kor-QnA
            - (corpus) https://huggingface.co/datasets/amphora/korfin-asc?row=3
        - 영어
            - (QA) FinLang/investopedia-instruction-tuning-dataset
            - (sentiment) Fingpt/fingpt-sentiment-train
    - pair 구성하기
        1. positive pair : QA set의 경우 Query - Answer
            - Answer 없는 경우 : Title - Passage, **BM25** 등
        2. Negative pair : in-batch + Hard negative
            - Teacher Model을 통해 Hard negative Mining 진행
            - Query당 최소 1개 ~ 최대 4개의 Hard negative (Batch / Base Model 크기에 따라 달라짐)
        
        **+) 금융 도메인 / 데이터 특성에 따라 pair 선정 방식 별도 추가**
        
3. Instruction Tuning
    - prefix 형식 {task_prefix} : {query} → 데이터셋에 따라 변경
    - Tuning 진행 (LoRA 적용)
    - A6000x4 + deepspeed stage 3 가 최소 사양
    
4. Evaluation
    1. (한국어) Eval dataset 구축 (KorFinMTEB)
    2. (영어) 현존하는 FinMTEB 평가

## 1. Dataset

### 한국어

- https://huggingface.co/datasets/BCCard/BCCard-Finance-Kor-QnA
- 수집한 데이터셋

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import json


In [1]:

bccard_df = pd.read_json("hf://datasets/BCCard/BCCard-Finance-Kor-QnA/bccard-finance-qna.jsonl", lines=True)
bccard_df.head()

c:\Users\hwyew\miniconda3\envs\finance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,instruction,output
0,비씨카드는 어떤 회사인가요?,"비씨카드(BC카드)는 1982년에 설립된 대한민국의 대표적인 카드사로, 주로 결제 ..."
1,비씨카드의 주요 서비스는 뭐야?,"비씨카드는 다양한 금융 서비스와 결제 솔루션을 제공하는 회사로, 특히 결제 프로세싱..."
2,비씨카드에서 처리하는 주요 금융 거래는 어떤 것들이 있나요?,비씨카드에서 처리하는 주요 금융 거래는 주로 신용카드와 체크카드의 결제 프로세싱입니...
3,비씨카드의 현황과 변화 요약을 해줘,"비씨카드는 한국의 대표적인 카드사로서, 주로 카드 결제 프로세싱에 강점을 가지고 있..."
4,비씨카드의 디지털 혁신 노력에는 어떤 것들이 있나요?,"비씨카드는 디지털 혁신의 일환으로 ""페이북""이라는 디지털 결제 플랫폼을 개발하여 운..."


In [ ]:
# bccard_df.shape

### 데이터 전처리


In [3]:
# # "비씨카드"가 포함된 행 제거 -- 최대한 일반적 문장
# bccard_df = bccard_df[~bccard_df['instruction'].str.contains("비씨카드")]
# bccard_df = bccard_df[~bccard_df['instruction'].str.contains("BC")]
# bccard_df = bccard_df[~bccard_df['instruction'].str.contains("GOAT")]
# bccard_df = bccard_df[~bccard_df['instruction'].str.contains("K-패스")]
# bccard_df = bccard_df[~bccard_df['instruction'].str.contains("에어로 아시아나")]
# bccard_df = bccard_df[~bccard_df['instruction'].str.contains("시발카드")]

In [ ]:
# bccard_df.reset_index(drop=True, inplace=True)
# bccard_df = bccard_df.rename(columns={"instruction" : "Query", "output" : "Answer"})
# bccard_df.head()

In [5]:
# bccard_df = bccard_df.sample(frac=1).reset_index(drop=True)
# bccard_df.head(30)

In [ ]:
# bccard_df_sample = bccard_df.sample(frac=0.1).reset_index(drop=True)
# bccard_df_sample

In [6]:
# bccard_df_sample.to_csv("./data/bccard_df_sample.csv", index=False)

## 2. Hard negative Mining
- by teacher model / BM25
- Query 당 4개
- config에 따라 0.95로 진행

### Teacher Model Embedding

- Teacher Model
    - bge-m3 (https://huggingface.co/BAAI/bge-m3) - multilingual
    - KURE-v1 (https://huggingface.co/nlpai-lab/KURE-v1) -> bge-m3 기반 (가장 최신)
    - e5-mistral-7b-instruct (영어 사용을 권장)


- base Model : multilingual-e5-large / BAAI-bge-m3

In [2]:
import json
from FlagEmbedding import BGEM3FlagModel
from sentence_transformers import SentenceTransformer


def make_embed_json(df, model_name, data_name, query_col_name="Query", pos_col_name="Answer", sample=True):
    data_list = []
    if model_name == "BAAI/bge-m3":
        model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True, cache="./cache")
    elif model_name == "nlpai-lab/KURE-v1":
        # Download from the 🤗 Hub
        model = SentenceTransformer("nlpai-lab/KURE-v1", cache_folder="./cache")
    else:
        raise ValueError("Wrong model name :", model_name)
        

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        if model_name == "BAAI/bge-m3":
            query_emb = model.encode(row[query_col_name], verbose=False)['dense_vecs'].tolist()   # 텍스트를 embedding
            answer_emb = model.encode(row[pos_col_name], verbose=False)['dense_vecs'].tolist()
        elif model_name == "nlpai-lab/KURE-v1":
            query_emb = model.encode(row[query_col_name]).tolist()   # 텍스트를 embedding
            answer_emb = model.encode(row[pos_col_name]).tolist() 

        data_list.append({
            "index": idx,
            "query_text":row[query_col_name],
            "query_embed": query_emb,
            "answer_text":row[pos_col_name],
            "answer_embed": answer_emb
        })

    # JSON으로 저장
    if sample==True:
        output_file = f"./data/{data_name}_{model_name.split('/')[1]}_sample.json"
    else:
        output_file = f"./data/{data_name}_{model_name.split('/')[1]}.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

    print(f"데이터가 {output_file}에 저장되었습니다.")


import torch
from torch.amp import autocast

import pandas as pd
from tqdm import tqdm
import numpy as np

from transformers.utils.logging import disable_progress_bar
disable_progress_bar()


def calculate(emb_1, emb_2):
    return emb_1 @ emb_2.T

# Hard Negative Mining -- batch 제거
def mine_hard_negatives(data, max_neg=4):
    results = []
    answer_embeds = [np.array(item['answer_embed']) for item in data]

    for row in tqdm(data, total=len(data)):
        torch.cuda.empty_cache()  # GPU 메모리 정리
        
        query = row['query_text']
        positive_answer = row['answer_text']

        query_emb = np.array(row['query_embed'])
        pos_emb = np.array(row['answer_embed'])

        # Positive 유사도 계산
        with torch.no_grad():
            pos_score = calculate(query_emb, pos_emb)

        # Threshold 계산
        max_neg_score_threshold = pos_score * 0.95

        # Negative 후보 생성
        negative_candidates = [item['answer_text'] for item in data if item['answer_text'] != positive_answer]
        negative_embeds = [np.array(item['answer_embed']) for item in data if item['answer_text'] != positive_answer]

        # Negative Scores를 배치로 계산
        hard_negatives = []

        with torch.no_grad():
            negative_scores = calculate(query_emb, np.stack(negative_embeds))

        # Hard Negative 후보 저장
        for neg, neg_score in zip(negative_candidates, negative_scores):
            if neg_score <= max_neg_score_threshold:
                hard_negatives.append((neg, neg_score))

        
        # 유사도가 높은 순으로 정렬
        hard_negatives = sorted(hard_negatives, key=lambda x: x[1], reverse=True)[:max_neg]

        # 결과 저장
        for neg, neg_score in hard_negatives:
            results.append({
                'Query': query,
                'Positive Answer': positive_answer,
                'Hard Negative': neg,
                'Positive Score': pos_score.item(),
                'Negative Score': neg_score.item()
            })

    return pd.DataFrame(results)

c:\Users\hwyew\miniconda3\envs\finance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### bge-m3

#### Sample

In [5]:
import pandas as pd

bccard_df_sample = pd.read_csv("./data/bccard_df_sample.csv")

In [ ]:
# sample
make_embed_json(bccard_df_sample, "BAAI/bge-m3", "BCCard")

In [ ]:
import json

with open("./data/BCCard_bge-m3_sample.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

bge_m3_sample = mine_hard_negatives(data_list)
bge_m3_sample.to_csv("./data/BCCard_bge-m3_sample_hard_negative.csv", index=False)

In [7]:
bge_m3_sample.to_csv("./data/BCCard_bge-m3_sample_hard_negative.csv", index=False)

#### Total

In [ ]:
# total
make_embed_json(bccard_df, "BAAI/bge-m3", "BCCard", sample=False)

In [ ]:
# JSON 파일 읽기
with open("./data/BCCard_bge-m3.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

bge_m3 = mine_hard_negatives(data_list)
bge_m3.to_csv("./data/BCCard_bge-m3_hard_negative.csv", index=False)

### KURE-v1

#### sample

In [ ]:
make_embed_json(df, model_name, data_name, query_col_name="Query", pos_col_name="Answer", sample=True):

In [ ]:
import pandas as pd
bccard_df_sample = pd.read_csv("./data/bccard_df_sample.csv")
bccard_df_sample.head()

In [ ]:
make_embed_json(bccard_df_sample, "nlpai-lab/KURE-v1", "BCCard")

In [ ]:
# JSON 파일 읽기
with open("./data/BCCard_KURE-v1_sample.json", "r", encoding="utf-8") as f:
    kure = json.load(f)

In [ ]:
kure = mine_hard_negatives(data_list)
kure.to_csv("./data/BCCard_KURE-v1_sample_hard_negative.csv", index=False)

#### total

In [3]:
import pandas as pd
bccard_df = pd.read_csv("./data/bccard_df.csv")
bccard_df.head()

,Query,Answer
0,R의 공포 시기에 카드사는 신용 등급이 낮은 고객에 대해 어떻게 대응하나요?,R의 공포 시기에 신용 등급이 낮은 고객에 대한 대응은 카드사에게 중요한 과제가 됩...
1,모기지 백트 증권의 만기는 어떻게 결정되나요?,모기지 백트 증권의 만기는 기초 자산인 주택담보대출의 만기에 따라 결정됩니다. 대출...
2,주식 거래 수수료가 면제되는 경우도 있나요?,"특정 조건을 만족할 경우 주식 거래 수수료가 면제되는 경우도 있습니다. 예를 들어,..."
3,기준금리 변동이 투자 심리에 미치는 영향은 무엇인가요?,기준금리 변동은 투자 심리에 큰 영향을 미칩니다. 금리 인상 시 대출 비용 증가와 ...
4,케이뱅크의 예금 상품은 어떤 것이 있나요?,"케이뱅크의 예금 상품으로는 자유 입출금 통장, 정기 예금, 적금 등이 있습니다. 각..."


In [4]:
bccard_df

,Query,Answer
0,R의 공포 시기에 카드사는 신용 등급이 낮은 고객에 대해 어떻게 대응하나요?,R의 공포 시기에 신용 등급이 낮은 고객에 대한 대응은 카드사에게 중요한 과제가 됩...
1,모기지 백트 증권의 만기는 어떻게 결정되나요?,모기지 백트 증권의 만기는 기초 자산인 주택담보대출의 만기에 따라 결정됩니다. 대출...
2,주식 거래 수수료가 면제되는 경우도 있나요?,"특정 조건을 만족할 경우 주식 거래 수수료가 면제되는 경우도 있습니다. 예를 들어,..."
3,기준금리 변동이 투자 심리에 미치는 영향은 무엇인가요?,기준금리 변동은 투자 심리에 큰 영향을 미칩니다. 금리 인상 시 대출 비용 증가와 ...
4,케이뱅크의 예금 상품은 어떤 것이 있나요?,"케이뱅크의 예금 상품으로는 자유 입출금 통장, 정기 예금, 적금 등이 있습니다. 각..."
...,...,...
31245,원자재 선물 거래의 기본 원리는 무엇인가요?,원자재 선물 거래의 기본 원리는 계약 당사자 간의 합의입니다. 구매자(매수자)는 정...
31246,배대지를 통해 구매한 물품의 환불 절차는 어떻게 되나요?,"배대지를 통해 구매한 물품의 환불 절차는 해당 쇼핑몰의 환불 정책을 확인하고, 배대..."
31247,대출 이자율이 높은 이유는 무엇인가요?,"대출 이자율은 차주의 신용도, 대출 상품의 종류, 경제 상황 등에 따라 달라집니다...."
31248,Light 할부를 취소할 수 있나요?,"네, Light 할부는 신청 후에도 취소가 가능합니다. 취소 절차는 간단하게 BC카..."


In [5]:
make_embed_json(bccard_df, "nlpai-lab/KURE-v1", "BCCard", sample=False)

100%|██████████| 31250/31250 [18:30<00:00, 28.14it/s]


데이터가 ./data/BCCard_KURE-v1.json에 저장되었습니다.


In [6]:
# JSON 파일 읽기
with open("./data/BCCard_KURE-v1.json", "r", encoding="utf-8") as f:
    kure = json.load(f)

In [8]:
kure = mine_hard_negatives(kure)
kure.to_csv("./data/BCCard_KURE-v1_hard_negative.csv", index=False)

100%|██████████| 31250/31250 [7:22:31<00:00,  1.18it/s]  


## 1-2. Dataset - Naver Finnews

In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import ast

In [ ]:
# sample
naver_news_sample = pd.read_csv("./data/news_sample.csv")
naver_news_sample.reset_index(drop=True, inplace=True)
naver_news_sample.head()

In [4]:
# total
naver_news = pd.read_csv("./data/naver_main_news_2024.csv")
naver_news.dropna(subset=['cleaned_text'], inplace=True)
naver_news['cleaned_text'] = naver_news['cleaned_text'].apply(lambda x : ' '.join(ast.literal_eval(x)))

naver_news.reset_index(drop=True, inplace=True)
naver_news_cleaned = pd.DataFrame()
naver_news_cleaned['Query'] = naver_news['title']
naver_news_cleaned['Answer'] = naver_news['cleaned_text']

naver_news_cleaned.head()

,Query,Answer
0,해외 10대 이슈-저성장시대[2024 신년기획],세계경제 먹구름. 사진연합뉴스 세계 경제가 고금리의 후유증을 앓고 있다. 미국 연방...
1,올해도 공모주 청약 이어진다…'몸값 조(兆) 단위' 기대주는?,작년 말 따따블 3개 종목 연달아 등장에이피알현대마린솔루션 눈길 올해 시장에서 몸...
2,지난해 증시 달군 ‘반·로·이’새해에도 날까…‘바이오’도 대기 중,2023년 국내 증시의 주도주 테마는 반로이 였다. 국내 주식형 상장지수펀드 수익률...
3,증권사 전문가 5명 중 4명 “상반기에 주식 사라”,언제 어떤 종목 투자할까 미국 뉴욕증권거래소에서 트레이더가 모니터를 바라보고 있다....
4,금리인하 시작된다…올해 상업용 부동산 유망 투자처는 어디?,지난해 고금리가 계속되면서 부동산 시장이 휘청였다. 주택뿐 아니라 국내 상업용 부동...


In [5]:
naver_news_cleaned

,Query,Answer
0,해외 10대 이슈-저성장시대[2024 신년기획],세계경제 먹구름. 사진연합뉴스 세계 경제가 고금리의 후유증을 앓고 있다. 미국 연방...
1,올해도 공모주 청약 이어진다…'몸값 조(兆) 단위' 기대주는?,작년 말 따따블 3개 종목 연달아 등장에이피알현대마린솔루션 눈길 올해 시장에서 몸...
2,지난해 증시 달군 ‘반·로·이’새해에도 날까…‘바이오’도 대기 중,2023년 국내 증시의 주도주 테마는 반로이 였다. 국내 주식형 상장지수펀드 수익률...
3,증권사 전문가 5명 중 4명 “상반기에 주식 사라”,언제 어떤 종목 투자할까 미국 뉴욕증권거래소에서 트레이더가 모니터를 바라보고 있다....
4,금리인하 시작된다…올해 상업용 부동산 유망 투자처는 어디?,지난해 고금리가 계속되면서 부동산 시장이 휘청였다. 주택뿐 아니라 국내 상업용 부동...
...,...,...
55512,"""코스피도 없던 시절""…45년 전 마지막 계엄령때 증시는 어땠나",윤석열 대통령이 간밤 비상계엄을 선포했다 국회에 막혀 계엄을 해제한 4일 오전 서울...
55513,증권가 “증시 단기 변동성 확대…외인 투매 등 주시”[실패한 계엄령],윤석열 대통령의 비상계엄령 선포 후 국회에서 비상계엄해제요 구안이 가결된 가운데 4...
55514,"""비상계엄령, 尹정부 핵심정책 '밸류업'에 불똥 튈 수도""",윤석열 대통령이 비상계엄을 선언했다 국회의 의결로 계엄을 해제한 4일 오전 서울 중...
55515,"""카카오만 왜 올라?"" 7% 치솟으며 시총 20조 돌파…그룹株 '들썩'[핫종목]",경기 성남시 분당구 카카오 판교아지트의 모습..뉴스1 1 카카오가 7% 넘...


### bge-m3

#### sample

In [ ]:
make_embed_json(naver_news_sample, "BAAI/bge-m3", "NaverNews")

In [11]:
# JSON 파일 읽기
with open("./data/NaverNews_bge-m3_sample.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

In [ ]:
bge_sample = mine_hard_negatives(data_list)
bge_sample.to_csv("./data/NaverNews_bge-m3_sample_hard_negative.csv", index=False)

#### total

In [8]:
make_embed_json(naver_news_cleaned, "BAAI/bge-m3", "NaverNews", sample=False)

100%|██████████| 55517/55517 [1:00:39<00:00, 15.25it/s]


데이터가 ./data/NaverNews_bge-m3.json에 저장되었습니다.


In [9]:
# JSON 파일 읽기
with open("./data/NaverNews_bge-m3.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

In [10]:
bge = mine_hard_negatives(data_list)
bge.to_csv("./data/NaverNews_bge-m3_hard_negative.csv", index=False)

100%|██████████| 55517/55517 [23:37:13<00:00,  1.53s/it]   


### KURE

#### sample

In [ ]:
make_embed_json(naver_news_sample, "nlpai-lab/KURE-v1", "NaverNews")

In [ ]:
# JSON 파일 읽기
with open("./data/NaverNews_KURE-v1_sample.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

In [ ]:
kure_sample = mine_hard_negatives(data_list)
kure_sample.to_csv("./data/NaverNews_KURE-v1_sample_hard_negative.csv", index=False)

#### total

In [14]:
make_embed_json(naver_news_cleaned, "nlpai-lab/KURE-v1", "NaverNews", sample=False)

100%|██████████| 55517/55517 [8:49:01<00:00,  1.75it/s]      


데이터가 ./data/NaverNews_KURE-v1.json에 저장되었습니다.


In [16]:
# JSON 파일 읽기
with open("./data/NaverNews_KURE-v1.json", "r", encoding="utf-8") as f:
    data_list = json.load(f)

In [17]:
kure = mine_hard_negatives(data_list)
kure.to_csv("./data/NaverNews_KURE-v1_hard_negative.csv", index=False)

100%|██████████| 55517/55517 [24:01:47<00:00,  1.56s/it]   
